### GibbsNew
One feature we get from the new sampler design is more flexibility in our Gibbs sampling.
Now we can for example run with sampler instances, which allows us to set parameters and also use all samplers in Gibbs sampling.

In [ ]:
import numpy as np
from cuqi.distribution import Gamma, Gaussian, GMRF, JointDistribution, LMRF
from cuqi.experimental.mcmc import NUTSNew, GibbsNew, ConjugateNew, LinearRTONew, ConjugateApproxNew, UGLANew
from cuqi.sampler import Gibbs, Conjugate, LinearRTO
from cuqi.testproblem import Deconvolution1D

In [ ]:
# Forward problem
A, y_data, info = Deconvolution1D(dim=128, phantom='sinc', noise_std=0.001).get_components()

In [ ]:
# Target
s = Gamma(1, 1e-4)
x = GMRF(np.zeros(A.domain_dim), 50)
y = Gaussian(A@x, lambda s: 1/s)

# %%
target = JointDistribution(y, x, s)(y=y_data)


In [ ]:
# Old-style Gibbs. Only supports "Static" (non-adaptive) samplers
sampling_strategy = {
    "x" : LinearRTO,
    "s" : Conjugate
}

sampler = Gibbs(target, sampling_strategy)

# %%

samples = sampler.sample(200, 50)

In [ ]:
samples["x"].plot_ci()

In [ ]:
# New-style Gibbs. Supports object-oriented that maintain state through iterations

sampling_strategy = {
    "x" : LinearRTONew(maxit=25), # <--- Notice instance of class with set parameters
    "s" : ConjugateNew()          # <--- Notice instance of class with default parameters
}

sampler = GibbsNew(target, sampling_strategy)

# Sample using new-style Gibbs
sampler.warmup(50)
sampler.sample(200)
samples = sampler.get_samples()


In [ ]:
samples["x"].plot_ci()

In [ ]:
# Try with NUTS
sampling_strategy = {
    "x" : NUTSNew(max_depth=10),
    "s" : ConjugateNew()
}

# Here we do 10 internal steps with NUTS for each Gibbs step
num_sampling_steps = {
    "x" : 10,
    "s" : 1
}

sampler = GibbsNew(target, sampling_strategy, num_sampling_steps)

sampler.warmup(50)
sampler.sample(200)
samples = sampler.get_samples()

In [ ]:
samples["x"].plot_ci()

In [ ]:
# Try with ConjugateApprox (LMRF prior)

# Forward problem
A, y_data, info = Deconvolution1D(dim=128, phantom='square', noise_std=0.001).get_components()

# Target
d = Gamma(1, 1e-4)
s = Gamma(1, 1e-4)
x = LMRF(0, lambda d: 1/d, geometry=A.domain_geometry)
y = Gaussian(A@x, lambda s: 1/s)

# Posterior
target = JointDistribution(y, x, s, d)(y=y_data)

# Sampling strategy
sampling_strategy = {
    "x" : UGLANew(),
    "s" : ConjugateNew(),
    "d" : ConjugateApproxNew()
}

# Gibbs sampler
sampler = GibbsNew(target, sampling_strategy)

# Run sampler
sampler.warmup(50)
sampler.sample(200)
samples = sampler.get_samples()

# Plot
samples["x"].plot_ci()